In [3]:
import sys, os, glob, time
import time
import h5py
from copy import copy as copy
import numpy as np
import scipy as sp
import numpy.random as rn
import matplotlib.pyplot as plt
from multiprocessing import Pool, Process, Manager

from parameters import data_parameters
from modules.density_extraction import density_extraction, calc_all_dists

#pip install corner

In [4]:
def log_prior(theta):
    d1_min, d1_max = 1e-4, 10.0
    d2_max, d2_max = 1e-4, 10.0
    d3 = np.sqrt((theta[:,0] - theta[:,1]*np.cos(theta[:,2]))**2
        + (theta[:,1]*np.sin(theta[:,2]))**2)
    prob = np.zeros(theta.shape[0])
    inds = (theta[:,2] > np.pi) + (theta[:,2] < 1e-3)\
          + (theta[:,0] < d1_min) + (theta[:,0] > d1_max)\
          + (theta[:,1] < d2_min) + (theta[:,1] > d2_max)
          #+ (theta[:,1] > d3) + (theta[:,0] > d3)           #

    log_prob[inds] = -1*np.inf

    return log_prob

In [5]:
def theta_to_cartesian(theta):
    st, ct = np.sin(theta[:,2]/2), np.cos(theta[:,2]/2)
    molecules = np.zeros((theta.shape[0], 3, 3))
    molecules[:,0,0] = theta[:,0]*ct
    molecules[:,0,2] = theta[:,0]*st
    molecules[:,2,0] = theta[:,1]*ct
    molecules[:,2,2] = -1*theta[:,1]*st

    return molecules

In [6]:
def initialize_walkers(params, molecule):
    d1 = np.linalg.norm(molecule[0] - molecule[1])
    d2 = np.linalg.norm(molecule[2] - molecule[1])
    ang = np.arccos(np.sum(
        (molecule[0] - molecule[1])*(molecule[2] - molecule[1]))/(d1*d2))
    dists = rnd.normal(0, 1, size=(params["Nwalkers"], 3)) 

    dists *= 0.005*np.array([[d1, d2, ang]])
    dists += np.array([[d1, d2, ang]])

    return dists 

In [12]:
def molecule_ensemble_generator(molecule):
    N = 51
    std_r = 0.01
    std_a = 0.02
    d1 = np.linalg.norm(molecule[0] - molecule[1])
    d2 = np.linalg.norm(molecule[2] - molecule[1])
    ang = np.arccos(np.sum(
        (molecule[0] - molecule[1])*(molecule[2] - molecule[1]))/(d1*d2))
    def Prob(x, m, s):
        return np.exp(-0.5*((x-m)/s)**2)/(s*np.sqrt(2*np.pi))


    d1_distribution_vals = np.linspace(d1-std_r*4, d1+std_r*4, N)
    d2_distribution_vals = np.linspace(d2-std_r*4, d2+std_r*4, N)
    ang_distribution_vals = np.linspace(ang-std_a*4, ang+std_a*4, N)

    d1_probs = Prob(d1_distribution_vals, d1, std_r)
    d1_probs /= np.sum(d1_probs)
    d2_probs = Prob(d2_distribution_vals, d2, std_r)
    d2_probs /= np.sum(d2_probs)
    ang_probs = Prob(ang_distribution_vals, ang, std_a)
    ang_probs /= np.sum(ang_probs)
    thetas = np.ones((N,N,N,3))*np.nan
    joint_probs = np.ones((N,N,N,1))*np.nan

    i1 = np.arange(N)
    for i in i1:
        thetas[i,:,:,0] = d1_distribution_vals[i]
        for j in i1:
            thetas[i,j,:,1] = d2_distribution_vals[j]
            thetas[i,j,i1,2] = ang_distribution_vals[i1]
            joint_probs[i,j,:,0] = d1_probs[i]*d2_probs[j]*ang_probs

    thetas = np.reshape(thetas, (-1,3))
    joint_probs = np.reshape(joint_probs, (-1,1))


    molecules = theta_to_cartesian(thetas)
    norm = np.sum(joint_probs)
    mean_thetas = np.sum(thetas*joint_probs, 0)/norm
    mean_cartesian = np.sum(np.expand_dims(joint_probs, -1)*molecules, 0)/norm
    mean_d3 = np.linalg.norm(np.sum((molecules[:,0] - molecules[:,-1])*joint_probs, 0)/norm)
    std_thetas = np.sqrt(np.sum((thetas - mean_thetas)**2*joint_probs, 0)/norm)
    std_cartesian = np.sqrt(np.sum(np.expand_dims(joint_probs, -1)*(molecules - mean_cartesian)**2, 0)/norm)
    std_d3 = np.sqrt(
      np.sum(joint_probs[:,0]*(np.linalg.norm(molecules[:,0]-molecules[:,-1], axis=-1)-mean_d3)**2, 0)/norm)

    print("\n#####  Mean molecule thetas  #####")
    print(mean_thetas)
    print("\n#####  STD molecule thetas  #####")
    print(std_thetas)
    print("\n#####  Mean molecule cartesian  #####")
    print(mean_cartesian)
    print("\n#####  STD molecule cartesian  #####")
    print(std_cartesian)
    print("\n#####  Mean d3  #####")
    print(mean_d3)
    print("\n#####  STD d3  #####")
    print(std_d3)
    print("\n")



    #print(np.sum(molecules*np.expand_dims(weights, -1), 0))
    return molecules, joint_probs

In [8]:
def run_mcmc(data_parameters,
    extraction=None, input_params=None):

  #####  Change Parameters Based on Input  #####
  if input_params is not None:
    for k,v in input_params.items():
      if k not in data_parameters:
        print("WARNING: parameter {} was not found in input data_parameters, not adding".format(k))

      data_parameters[k] = v

  #####  Create extraction Class if not given  #####
  if extraction is None:
      extraction = density_extraction(data_parameters,
          log_prior=log_prior,
          theta_to_cartesian=theta_to_cartesian,
          ensemble_generator=molecule_ensemble_generator,
          get_ADMs=get_ADMs)
    molecule = extraction.setup_calculations()


  ######################
  #####  Run MCMC  #####
  ######################

    walkers_init = initialize_walkers(
        data_parameters, extraction.atom_positions)

  extraction.run_mcmc(walkers_init, data_parameters["run_limit"])

  chain = extraction.get_mcmc_results()

IndentationError: unindent does not match any outer indentation level (<ipython-input-8-c00bf9ef49e1>, line 19)

# Run Analysis

## Setup Density Extraction Environment, Calculations, Dataset (simulated)

In [13]:
extraction = density_extraction(data_parameters,
    log_prior=log_prior,
    theta_to_cartesian=theta_to_cartesian,
    ensemble_generator=molecule_ensemble_generator,
    get_ADMs=None)
molecule = extraction.setup_calculations()

['O', '0.000000', '0.465300', '1.098900']
['N', '0.000000', '0.000000', '0.000000']
['O', '0.000000', '0.465300', '-1.098900']
INP DOM (500,)

#####  Mean molecule thetas  #####
[1.19335045 1.19335045 2.34052362]

#####  STD molecule thetas  #####
[0.0099961  0.0099961  0.01999221]

#####  Mean molecule cartesian  #####
[[ 0.46527675  0.          1.0988451 ]
 [ 0.          0.          0.        ]
 [ 0.46527675  0.         -1.0988451 ]]

#####  STD molecule cartesian  #####
[[0.01165552 0.         0.01031316]
 [0.         0.         0.        ]
 [0.01165552 0.         0.01031316]]

#####  Mean d3  #####
2.1976901983449695

#####  STD d3  #####
0.015999685037020782




MemoryError: Unable to allocate array with shape (6, 3, 500, 132651) and data type float64

## Setup and Run the MCMC Sampler

In [ ]:
walkers_init = initialize_walkers(
    data_parameters, extraction.atom_positions)

extraction.run_mcmc(walkers_init, data_parameters["run_limit"])



## Gather MCMC Results

In [ ]:
chain = extraction.get_mcmc_results()